In [17]:
# import pandas as pd

# df = pd.read_csv('Processed_Headings/normalized_section_data 2.csv')
# df.head()

In [18]:
# df = df.dropna(subset=['sec-norm'])

In [19]:
# df_grouped = df.groupby(["PMCID", "PMID"]).agg(list)
# df_grouped

In [20]:
# import pandas as pd

# def convert_to_sep_format(row):
#     titles = row["sec-title"]  # 取得標題列表
#     labels = row["sec-norm"]   # 取得標籤列表

#     # 插入 `[SEP]`
#     inputs_with_sep = []
#     labels_with_sep = []

#     for i, title in enumerate(titles):
#         inputs_with_sep.append(title)
#         labels_with_sep.append("O")  # 一般 token 標記為 "O"
        
#         # 在每個標題後面插入 [SEP]，對應標籤來自 sec-norm
#         if i < len(labels):  # 確保標籤數量正確
#             inputs_with_sep.append("[SEP]")
#             labels_with_sep.append(labels[i])  # 讓 [SEP] 位置承擔標籤

#     return pd.Series([inputs_with_sep, labels_with_sep])

# # 對 DataFrame 進行轉換
# df_grouped[["inputs_with_sep", "labels_with_sep"]] = df_grouped.apply(convert_to_sep_format, axis=1)

# df_grouped


In [21]:
# df_grouped = df_grouped.drop(columns=['sec-type', 'sec-title', 'sec-norm'])
# df_grouped.to_csv('Processed_Headings/grouped.csv')

In [23]:
import pandas as pd
df_grouped = pd.read_csv('Processed_Headings/grouped.csv')


In [24]:
df_grouped

,PMCID,PMID,inputs_with_sep,labels_with_sep
0,PMC10000014,36921129,"['INTRODUCTION', '[SEP]', 'METHODS', '[SEP]', ...","['O', 'background', 'O', 'methods', 'O', 'resu..."
1,PMC10000015,36909079,"['Introduction', '[SEP]', 'Case presentation',...","['O', 'background', 'O', 'methods', 'O', 'conc..."
2,PMC10000017,36921128,"['INTRODUCTION', '[SEP]', 'METHODS', '[SEP]', ...","['O', 'background', 'O', 'methods', 'O', 'resu..."
3,PMC10000020,34107125,"['Introduction', '[SEP]', 'Results', '[SEP]', ...","['O', 'background', 'O', 'results', 'O', 'resu..."
4,PMC10000021,36583948,"['INTRODUCTION', '[SEP]', 'MATERIALS AND METHO...","['O', 'background', 'O', 'methods', 'O', 'resu..."
...,...,...,...,...
4654684,PMC9999731,36911530,"['Introduction', '[SEP]', 'Materials and metho...","['O', 'background', 'O', 'methods', 'O', 'resu..."
4654685,PMC9999732,34391118,"['Introduction', '[SEP]', 'Methods', '[SEP]', ...","['O', 'background', 'O', 'methods', 'O', 'resu..."
4654686,PMC9999914,36936403,"['Introduction', '[SEP]', 'Methods', '[SEP]', ...","['O', 'background', 'O', 'methods', 'O', 'resu..."
4654687,PMC9999942,36920160,"['Introduction', '[SEP]', 'Conclusions', '[SEP]']","['O', 'background', 'O', 'conclusions']"


In [25]:
from torch.utils.data import DataLoader
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

MAX_LENGTH = 20  # 你希望的最大序列長度

def collate_fn(batch):
    """
    自定義 collate_fn，對 batch 進行動態 padding，並確保不超過 max_length
    """
    input_ids = [b[0].tolist()[:MAX_LENGTH] for b in batch]  # 確保不超過 MAX_LENGTH
    attention_mask = [b[1].tolist()[:MAX_LENGTH] for b in batch]
    labels = [b[2].tolist()[:MAX_LENGTH] for b in batch]

    # 找到 batch 中的最大長度
    batch_max_len = min(MAX_LENGTH, max(len(seq) for seq in input_ids))

    # Padding
    tokenizer_pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id else 0
    input_ids_padded = torch.tensor([seq + [tokenizer_pad_id] * (batch_max_len - len(seq)) for seq in input_ids])
    attention_mask_padded = torch.tensor([seq + [0] * (batch_max_len - len(seq)) for seq in attention_mask])
    labels_padded = torch.tensor([seq + [-1] * (batch_max_len - len(seq)) for seq in labels])  # -1 for ignored labels in CRF

    return input_ids_padded, attention_mask_padded, labels_padded


In [26]:
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

# 載入 BERT Tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 建立標籤映射表

label2id = {"O": 0, "objective": 1, "background": 2, "methods": 3, "results": 4, "conclusions": 5}
# label2id = defaultdict(int, label2id)
id2label = {v: k for k, v in label2id.items()}

class SectionDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=MAX_LENGTH):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Tokenize input
        encoded = self.tokenizer(['[CLS]'] + eval(row["inputs_with_sep"]), 
                                 is_split_into_words=True, 
                                 padding="max_length", 
                                 truncation=True, 
                                 max_length=self.max_length, 
                                 return_tensors="pt")

        input_ids = encoded["input_ids"].squeeze(0)
        attention_mask = encoded["attention_mask"].squeeze(0)

        # 轉換標籤為數字 index，並填充 padding
        labels = [0]+ [label2id[label] for label in eval(row["labels_with_sep"]) if label in label2id]
        labels = labels + [-1] * (self.max_length - len(labels))  # 用 -1 填充
        labels = torch.tensor(labels)
        
        # print(len(input_ids), len(attention_mask), len(labels))
        return input_ids, attention_mask, labels

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_grouped[:len(df_grouped)//40], test_size=0.1, random_state=49, shuffle=True)

train_dataset = SectionDataset(train_df, tokenizer)
test_dataset = SectionDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)



In [27]:
sample = train_dataset[0]
print(sample)

(tensor([  101,   101,  4955,   102,  4118,   102,  3463,   102,  6594,   102,
        27024,  3430,   102,   102,     0,     0,     0,     0,     0,     0]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]), tensor([ 0,  0,  2,  0,  3,  0,  4,  0,  5,  0,  2, -1, -1, -1, -1, -1, -1, -1,
        -1, -1]))


In [26]:
# !pip uninstall TorchCRF
# !pip install transformers==4.30.2


In [28]:
import torch.nn as nn
from transformers import BertModel
from torchcrf import CRF

class BERT_CRF(nn.Module):
    def __init__(self, num_labels):
        super(BERT_CRF, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.hidden_dim = self.bert.config.hidden_size
        self.fc = nn.Linear(self.hidden_dim, num_labels)  # 轉換為標籤 logits
        self.crf = CRF(num_labels, batch_first=True)  # CRF 層

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # (batch_size, seq_len, hidden_dim)
        emissions = self.fc(sequence_output)  # (batch_size, seq_len, num_labels)

        if labels is not None:
            loss = -self.crf(emissions, labels, mask=attention_mask.byte(), reduction="mean")
            return loss
        else:
            predictions = self.crf.decode(emissions, mask=attention_mask.byte())
            return predictions


In [ ]:
import torch.optim as optim
from tqdm import tqdm

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERT_CRF(num_labels=len(label2id)).to(device)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        loss = model(input_ids, attention_mask, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")


Epoch 1: 100%|██████████| 1048/1048 [03:12<00:00,  5.43it/s]


Epoch 1/2, Loss: 824.9436


Epoch 2: 100%|██████████| 1048/1048 [03:12<00:00,  5.44it/s]

Epoch 2/2, Loss: 202.8295


In [30]:
from sklearn.metrics import precision_recall_fscore_support

model.eval()
all_true_labels = []
all_pred_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        predictions = model(input_ids, attention_mask)

        for pred, true, tokens in zip(predictions, labels, input_ids):
            pred_labels = []
            true_labels = []

            for idx, (pred_idx, true_idx, token) in enumerate(zip(pred, true, tokens)):
                word = tokenizer.decode([token.item()])
                if word == "[SEP]" and true_idx.item() != -1:
                    pred_labels.append(id2label[pred_idx])
                    true_labels.append(id2label[true_idx.item()])


            if len(pred_labels) == len(true_labels):  # 確保長度匹配
                all_pred_labels.extend(pred_labels)
                all_true_labels.extend(true_labels)

print(f"True Labels: {len(all_true_labels)}, Pred Labels: {len(all_pred_labels)}")
print(f"Unique True Labels: {set(all_true_labels)}")
print(f"Unique Pred Labels: {set(all_pred_labels)}")

precision, recall, f1, _ = precision_recall_fscore_support(
    all_true_labels, all_pred_labels, average="macro"
)

print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


True Labels: 35241, Pred Labels: 35241
Unique True Labels: {'methods', 'conclusions', 'objective', 'results', 'background', 'O'}
Unique Pred Labels: {'methods', 'conclusions', 'results', 'background', 'O'}
Test Precision: 0.7844
Test Recall: 0.7812
Test F1 Score: 0.7820


/opt/packages/anaconda3-2024.10-1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
from sklearn.metrics import accuracy_score
precision, recall, f1, _ = precision_recall_fscore_support(
    all_true_labels, all_pred_labels, average="weighted"
)

print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

print(f"Test Accuracy: {accuracy_score(all_true_labels, all_pred_labels)}")

Test Precision: 0.9928
Test Recall: 0.9923
Test F1 Score: 0.9923
Test Accuracy: 0.9922817173178967


/opt/packages/anaconda3-2024.10-1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
